Vytvorenie grafu

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pylab
from matplotlib.colors import LinearSegmentedColormap



In [ ]:
graph = {}
nodes = set()
edges = []
with open('linky.txt','r') as file:
    for line in file:
        stanice = line.split(':  ')[1].strip('\n').split(';')
        linka =  line.split(':  ')[0]
        print(linka)
        for i,stanica in enumerate(stanice[:-1]):

            nodes.add(stanica)
            edges.append((stanice[i-1],stanica))
            if i != 0:
                try:
                    graph[stanice[i-1]][stanica] += 1
                except KeyError:
                    try:
                        graph[stanice[i-1]][stanica] = 1
                    except KeyError:
                        graph[stanice[i-1]] = {}
                        graph[stanice[i-1]][stanica] = 1

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 9: invalid continuation byte

Základné info

In [ ]:
n = len(nodes)
m = len(edges)
print(n,m)

In [ ]:
for g in graph:
    for r in graph[g]:
        if g == r:
            print(g)

In [ ]:
G = nx.DiGraph()
for node in graph:
    for neigbour in graph[node]:
        G.add_edges_from([(node,neigbour)], weight = graph[node][neigbour])

In [ ]:
G.remove_edges_from(nx.selfloop_edges(G))


In [ ]:
G2 = nx.MultiDiGraph()
for source, targets in graph.items():
    for target, weight in targets.items():
        for _ in range(weight):  # Add multiple edges if weight > 1
            G2.add_edge(source, target)

In [ ]:
nx.closeness_centrality(G2)

In [ ]:
print(nx.is_strongly_connected(G))
nx.number_strongly_connected_components(G)
nx.closeness_centrality(G)
a = dict(nx.shortest_path_length(G))
centrality = {}
for station in a:
    sum_dist = 0
    for neigbour in a[station]:
        sum_dist += a[station][neigbour]
    centrality[station] = sum_dist/n

minimal = float("inf")
maximal = 0
uzol = ""
koniec = ""
for station in centrality:
    if centrality[station] < minimal:
        minimal = centrality[station]
        uzol = station
    if centrality[station] > maximal:
        maximal = centrality[station]
        koniec = station
print(uzol, minimal)
print(koniec, maximal)
print(nx.closeness_centrality(G)['Račianske mýto'])
print(nx.closeness_centrality(G)['Hlavná stanica'])
print(nx.closeness_centrality(G)['Danubiana'])
print(nx.betweenness_centrality(G))
print(nx.degree_centrality(G))
print("artikulacie", list(nx.articulation_points(G.to_undirected())))
print("mosty", list(nx.bridges(G.to_undirected())))

In [ ]:

edge_labels=dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])
dc = nx.degree_centrality(G)
node_importance_values = [dc[node] * (n-1) for node in G.nodes()]
print(node_importance_values)

In [ ]:

colors = [(0, 'blue'), (0.5, 'green'), (0.75, 'yellow'), (1, 'red')]
custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)

# Calculate closeness centrality
closeness_centrality = nx.closeness_centrality(G)

# Create a colormap based on closeness centrality values
node_colors = [closeness_centrality[node] for node in G.nodes()]

plt.figure(figsize=(100, 100))

pos = nx.kamada_kawai_layout(G)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif', font_color='black')

# Draw the graph with node colors based on closeness centrality
nodes = nx.draw(G, pos, node_color=node_colors, cmap=custom_cmap, node_size=[(np.log(val))*val * 50 for val in node_importance_values], edge_cmap=plt.cm.Reds)

# Create a dummy mappable object for the colorbar
dummy_mappable = plt.cm.ScalarMappable(cmap=custom_cmap)
dummy_mappable.set_array([])  # Set an empty array

# Add color bar
cbar = plt.colorbar(dummy_mappable)
cbar.set_label('Closeness Centrality')
plt.savefig("pokus.png")
#plt.show()

Bipartitný graf

In [ ]:
G_druhy = nx.DiGraph()
line_counters = {}

with open('linky.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(":")
        link_number = parts[0].strip()
        stops = [stop.strip() for stop in parts[1].split(";")][:-1]
        # Ked uz jednu linku "4" mame, pridame k tej druhej "A" aby sme ich vedeli rozlisit
        if link_number in line_counters:
            line_label = f"{link_number}{chr(64 + line_counters[link_number])}"
            line_counters[link_number] += 1
        else:
            line_label = link_number
            line_counters[link_number] = 1
        # Pridanie hrán medzi zastávkami v rámci linky
        for i in range(len(stops) - 1):
            G_druhy.add_edge(stops[i], stops[i+1], line=line_label)
        print(line_label)
        print(stops)

In [ ]:
# Nájdenie najkratšej cesty
def find_shortest_path_with_transfers(graph, start, end):
    try:
        shortest_path = nx.shortest_path(graph, start, end)
        return shortest_path
    except nx.NetworkXNoPath:
        return None

# Nájdenie najkratšej cesty
start_stop = "Nad lúčkami"
end_stop = "Cintorín Slávičie"
shortest_path = find_shortest_path_with_transfers(G_druhy, start_stop, end_stop)

# Výpis najkratšej cesty
print("Najkratšia cesta:")
if shortest_path:
    print(shortest_path)
else:
    print("Cesta nebola nájdená.")

In [ ]:
start_stop = "Nad lúčkami"
end_stop = "Cintorín Slávičie"
shortest_path = find_shortest_path_with_transfers(G_druhy, start_stop, end_stop)

# Výpis najkratšej cesty
print("Najkratšia cesta:")
if shortest_path:
    print(shortest_path)
else:
    print("Cesta nebola nájdená.")

Najkratšia cesta pre každú dvojicu

In [ ]:
transfers = {node: {} for node in G_druhy.nodes()}

In [ ]:
for node1 in G_druhy.nodes():
    for node2 in G_druhy.nodes():
        if node1 != node2:
            value = len(find_shortest_path_with_transfers(G_druhy, node1, node2))
            transfers[node1][node2] = value
            transfers[node2][node1] = value

In [ ]:
find_shortest_path_with_transfers(G_druhy, "STU", "STU")

Môj pokus o bipartitný graf

In [ ]:
Gb = nx.Graph()
line_counters = {}

with open('linky.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(":")
        link_number = parts[0].strip()
        stops = [stop.strip() for stop in parts[1].split(";")][:-1]
        #print(stops)
        Gb.add_node(link_number, bipartite = 0)
        Gb.add_nodes_from(stops, bipartite = 1)
        for stop in stops:
            Gb.add_edge(link_number, stop)



In [ ]:
nx.is_connected(Gb)
#Gb.nodes()
#connected_components = list(nx.connected_components(Gb))
#len(connected_components)
#connected_components
plt.figure(figsize=(50, 50))
pos = nx.bipartite_layout(Gb, nx.bipartite.sets(Gb)[0])
nx.draw(Gb, pos=pos, with_labels=True)
#plt.show()

In [ ]:
least_transfer = {station: {} for station in G.nodes()}
for station1 in G.nodes():
    for station2 in G.nodes():
        if station1 != station2:
            path = find_shortest_path_with_transfers(Gb, station1, station2)
            least_transfer[station1][station2] = path
            least_transfer[station2][station1] = path[::-1]
